In [68]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

from typing import Any, Optional, List, Dict
from collections import defaultdict
import nest_asyncio
nest_asyncio.apply()

import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2

import time
import tabulate
from qubx import lookup, logger
from qubx.core.basics import Position, ZERO_COSTS
from qubx.core.loggers import PositionsDumper, LogsWriter
from qubx.core.series import time_as_nsec
from qubx.core.utils import time_to_str, time_delta_to_str, recognize_timeframe
from qubx.utils.misc import makedirs 

 >  [dev] installing cython rebuilding hook
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
positions = [
    Position(lookup.find_symbol('BINANCE', s), ZERO_COSTS) for s in ['BTCUSDT', 'ETHUSDT']
] 

positions[0].change_position_by(0, 0.05, 63000)
positions[1].change_position_by(0, 0.5, 3200)

0

In [63]:
class PositionsWriter(LogsWriter):

    def write_data(self, log_type, data: List[Dict[str, Any]]):
        table = defaultdict(list)
        total_pnl, total_mkv = 0, 0
        
        for r in data:
            table['Symbol'].append(r['instrument_id'])
            table['Time'].append(r['timestamp'])
            table['Quantity'].append(r['quantity'])
            table['AvgPrice'].append(r['avg_position_price'])
            table['LastPrice'].append(r['current_price'])
            table['PnL'].append(r['pnl_quoted'])
            table['MarketValue'].append(r['market_value_quoted'])
            total_pnl += r['pnl_quoted']
            total_mkv += r['market_value_quoted']

        table['Symbol'].append('TOTAL')
        table['PnL'].append(total_pnl)
        table['MarketValue'].append(total_mkv)

        # - write to database table here
        print(f" ::: Strategy {self.strategy_id} @ {self.account_id} :::")
        print(tabulate.tabulate(table, ['Symbol', 'Time', 'Quantity', 'AvgPrice', 'LastPrice', 'PnL', 'MarketValue'], tablefmt='rounded_grid'))

In [64]:
pos_dumper = PositionsDumper(PositionsWriter('test', 'strategy1'), '5Sec')
pos_dumper.attach_positions(*positions)

In [ ]:
for _ in range(30):
    t = pd.Timestamp('now').asm8
    for p in positions:
        p.update_market_price(t, p.last_update_price + 10, 1)
    pos_dumper.store(t)
    time.sleep(0.5)

In [ ]:
pos_dumper.store(pd.Timestamp('now').asm8)

In [141]:
pd.read_csv('logs/FlipFlopStrat_binance-mde_positions.csv', index_col=['instrument_id'], parse_dates=['timestamp'])

,timestamp,pnl_quoted,quantity,realized_pnl_quoted,avg_position_price,current_price,market_value_quoted
instrument_id,,,,,,,
PEPEUSDT,2024-04-23 12:19:56.913859,-0.00380,380000.00,0.0,0.000007,0.000007,2.591600
SHIBUSDT,2024-04-23 12:19:56.913859,0.00000,0.00,0.0,0.000000,0.000026,0.000000
XVGUSDT,2024-04-23 12:19:56.913859,0.00000,0.00,0.0,0.000000,NaN,0.000000
BOMEUSDT,2024-04-23 12:19:56.913859,-6.72122,3014.00,0.0,0.012421,0.010191,30.715674
CKBUSDT,2024-04-23 12:19:56.913859,0.00000,0.00,0.0,0.000000,0.018586,0.000000
GALAUSDT,2024-04-23 12:19:56.913859,-4.25568,429.00,0.0,0.058230,0.048310,20.724990
LINKUSDT,2024-04-23 12:19:56.913859,-2.75210,1.45,0.0,17.150000,15.252000,22.115400


In [146]:
pd.read_csv('logs/FlipFlopStrat_binance-mde_portfolio.csv', index_col=['timestamp'], parse_dates=['timestamp'])

,instrument_id,pnl_quoted,quantity,realized_pnl_quoted,avg_position_price,current_price,market_value_quoted,exchange_time,commissions_quoted
timestamp,,,,,,,,,
2024-04-23 11:30:00,PEPEUSDT,0.000000,0.00,0.0,0.000000,0.000007,0.000000,2024-04-23T11:31:55.887639000,0.000000
2024-04-23 11:30:00,SHIBUSDT,-0.038000,380000.00,0.0,0.000027,0.000026,10.062400,2024-04-23T11:31:55.887639000,0.010100
2024-04-23 11:30:00,XVGUSDT,0.000000,0.00,0.0,0.000000,NaN,0.000000,2024-04-23T11:31:55.887639000,0.000000
2024-04-23 11:30:00,BOMEUSDT,-6.724234,3014.00,0.0,0.012421,0.010190,30.712660,2024-04-23T11:31:55.887639000,0.037439
2024-04-23 11:30:00,CKBUSDT,0.000000,0.00,0.0,0.000000,0.019035,0.000000,2024-04-23T11:31:55.887639000,0.000000
2024-04-23 11:30:00,GALAUSDT,-4.255680,429.00,0.0,0.058230,0.048310,20.724990,2024-04-23T11:31:55.887639000,0.024985
2024-04-23 11:30:00,LINKUSDT,-2.656400,1.45,0.0,17.150000,15.318000,22.211100,2024-04-23T11:31:55.887639000,0.024868
2024-04-23 11:55:00,PEPEUSDT,0.000000,0.00,0.0,0.000000,0.000007,0.000000,2024-04-23T11:57:55.105707000,0.000000
2024-04-23 11:55:00,SHIBUSDT,0.000000,0.00,0.0,0.000000,0.000026,0.000000,2024-04-23T11:57:55.105707000,0.000000


In [145]:
pd.read_csv('logs/FlipFlopStrat_binance-mde_executions.csv', index_col=['timestamp'], parse_dates=['timestamp'])

,instrument_id,side,filled_qty,price,commissions,commissions_quoted
timestamp,,,,,,
2024-04-23 11:34:03.684,SHIBUSDT,sell,-380000.0,0.000026,0.000012,BNB
2024-04-23 11:58:52.790,PEPEUSDT,buy,380000.0,0.000007,0.000003,BNB
2024-04-23 11:59:18.374,PEPEUSDT,sell,-380000.0,0.000007,0.000003,BNB
2024-04-23 12:10:59.012,PEPEUSDT,buy,380000.0,0.000007,0.000003,BNB
2024-04-23 12:20:02.843,PEPEUSDT,sell,-380000.0,0.000007,0.000003,BNB
